**Table of contents**<a id='toc0_'></a>    
- [image based visual servo(IBVS)](#toc1_)    
- [pybulletの起動](#toc2_)    
- [pybulletの初期設定](#toc3_)    
- [ロボットアームの生成](#toc4_)    
- [ARマーカが張り付けられたボックスを生成](#toc5_)    
- [光源位置の変更](#toc6_)    
- [使用関数の定義](#toc7_)    
- [カメラのパラメータ設定](#toc8_)    
- [ARマーカに関するパラメータの設定](#toc9_)    
- [ロボットアームのパラメータなどの設定](#toc10_)    
- [ビジュアルサーボの実行](#toc11_)    
- [（おまけ）特徴点位置の描画](#toc12_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[image based visual servo(IBVS)](#toc0_)

本notebookでは6軸のロボットアームを用いてimage based visual servoを行う方法を紹介します。

ビジュアルサーボ参考：https://github.com/RiddhimanRaut/Ur5_Visual_Servoing/blob/master/ur5_control_nodes/src/vs_ur5.py


（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[pybulletの初期設定](#toc0_)

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
# pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定（今回は、簡単のため重力は考慮しないこととする）
time_step = 1./240.
pybullet.setTimeStep(time_step) # 1stepあたりに経過する時間の設定

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[ロボットアームの生成](#toc0_)

In [3]:
# ロボットの読み込み
arm_start_pos = [0, 0, 0]  # 初期位置(x,y,z)を設定
arm_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
arm_id = pybullet.loadURDF("../urdf/simple6d_arm_with_force_sensor.urdf",arm_start_pos, arm_start_orientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
camera_distance = 1.5
camera_yaw = 50.0 # deg
camera_pitch = -40 # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[ARマーカが張り付けられたボックスを生成](#toc0_)

In [4]:
# 一面にARマーカが描かれたボックスの生成
box_bid = pybullet.loadURDF("../urdf/ar_marker_box.urdf", [0, 0, 0.5], useFixedBase=True)

# textureを設定(urdfファイルで指定したものと、同じものを指定)
texture_id = pybullet.loadTexture("../texture/ar_marker_box.png")
pybullet.changeVisualShape(box_bid, -1, textureUniqueId=texture_id)

# ARマーカが張り付けられたボックスの位置を調整するスライダーを設定 
pybullet.addUserDebugParameter("obj_x", -4, 4, 0.0)
pybullet.addUserDebugParameter("obj_y", -2, 0, -1.5)
pybullet.addUserDebugParameter("obj_z", -4, 8, 1.3)
pybullet.addUserDebugParameter("obj_roll", -3.14, 3.14, 0.0)
pybullet.addUserDebugParameter("obj_pitch", -3.14, 3.14, 0)
pybullet.addUserDebugParameter("obj_yaw", -3.14, 3.14, 0)

5

# <a id='toc6_'></a>[光源位置の変更](#toc0_)

In [5]:
# デフォルトの光源位置だと、ARマーカが認識されにくいので、光源位置を変更
pybullet.configureDebugVisualizer(lightPosition=[0, 0, 10])

# <a id='toc7_'></a>[使用関数の定義](#toc0_)

In [6]:
import numpy as np
import cv2

def Rx(theta):
    """
    x軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        x軸周りの回転行列
    """
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def Ry(theta):
    """
    y軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        y軸周りの回転行列
    """
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def Rz(theta):
    """
    z軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        z軸周りの回転行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])


def Hx(theta):
    """
    x軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        x軸回りの同次変換行列
    """
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hy(theta):
    """
    y軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        y軸回りの同次変換行列
    """
    return np.array([[np.cos(theta), 0, np.sin(theta), 0],
                     [0, 1, 0, 0],
                     [-np.sin(theta), 0, np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hz(theta):
    """
    z軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        z軸回りの同次変換行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                     [np.sin(theta), np.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def Hp(x, y, z):
    """
    平行移動の同次変換行列を求める

    Parameters
    ----------
    x : float
        x方向の移動量
    y : float
        y方向の移動量
    z : float
        z方向の移動量

    Returns
    -------
    numpy.ndarray
        平行移動の同次変換行列
    """
    return np.array([[1, 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]])
                     
def make_6d_jacobian_matrix(R, o1, o2, o3, o4, o5, o6, oc):
    """
    6軸ロボットアームの基礎ヤコビ行列を求める

    Parameters
    ----------
    R : numpy.ndarray
        ロボットアームの姿勢行列の各座標系間の回転行列
    o1 : numpy.ndarray
        リンク1の原点
    o2 : numpy.ndarray
        リンク2の原点
    o3 : numpy.ndarray
        リンク3の原点
    o4 : numpy.ndarray
        リンク4の原点
    o5 : numpy.ndarray
        リンク5の原点
    o6 : numpy.ndarray
        リンク6の原点
    oc : numpy.ndarray
        カメラの原点
   
        
    Returns
    -------
    Jv : numpy.ndarray
         基礎ヤコビ行列
    """

    
    R12 = R[0]
    R23 = R[1]
    R34 = R[2]
    R45 = R[3]
    R56 = R[4]
    R6C = R[5]
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])

    # ヤコビ行列の各要素を計算
    a1 = R12 @ ez
    a2 = R12 @ R23 @ ey
    a3 = R12 @ R23 @ R34 @ ey
    a4 = R12 @ R23 @ R34 @ R45 @ ez
    a5 = R12 @ R23 @ R34 @ R45 @ R56 @ ey
    a6 = R12 @ R23 @ R34 @ R45 @ R56 @ R6C @ ez

    # Ji = [[ai x (oc - oi)], 
    #       [ai]]
    j1 = np.concatenate((np.cross(a1, oc-o1).reshape(3, 1), a1.reshape(3, 1)), axis=0)
    j2 = np.concatenate((np.cross(a2, oc-o2).reshape(3, 1), a2.reshape(3, 1)), axis=0)
    j3 = np.concatenate((np.cross(a3, oc-o3).reshape(3, 1), a3.reshape(3, 1)), axis=0)
    j4 = np.concatenate((np.cross(a4, oc-o4).reshape(3, 1), a4.reshape(3, 1)), axis=0)
    j5 = np.concatenate((np.cross(a5, oc-o5).reshape(3, 1), a5.reshape(3, 1)), axis=0)
    j6 = np.concatenate((np.cross(a6, oc-o6).reshape(3, 1), a6.reshape(3, 1)), axis=0)

    Jv = np.concatenate((j1, j2, j3, j4, j5, j6), axis=1)
    return Jv

def make_4features_image_jacobian_matrix(current_feature_points, camera_to_obj, f):
    """
    4つの特徴点の画像ヤコビ行列を求める

    Parameters
    ----------
    current_feature_points : numpy.ndarray
        現在の特徴点座標
    camera_to_objs : float
        カメラからオブジェクトまでの距離
    f : float
        カメラの焦点距離
    
    Returns
    -------
    Ji : numpy.ndarray
        画像ヤコビ行列
    """
    # 4つの特徴点の画像ヤコビ行列
    u1 = current_feature_points[0]
    v1 = current_feature_points[1]
    u2 = current_feature_points[2]
    v2 = current_feature_points[3]
    u3 = current_feature_points[4]
    v3 = current_feature_points[5]
    u4 = current_feature_points[6]
    v4 = current_feature_points[7]

    Ji = np.array([[-f*(1/camera_to_obj), 0, u1*(1/camera_to_obj), u1*v1/f, -(f+(1/f)*u1**2), v1],
                   [0, -f*(1/camera_to_obj), v1*(1/camera_to_obj), f+(1/f)*v1**2, -(1/f)*u1*v1, -u1],
                   [-f*(1/camera_to_obj), 0, u2*(1/camera_to_obj), u2*v2/f, -(f+(1/f)*u2**2), v2],
                   [0, -f*(1/camera_to_obj), v2*(1/camera_to_obj), f+(1/f)*v2**2, -(1/f)*u2*v2, -u2],
                   [-f*(1/camera_to_obj), 0, u3*(1/camera_to_obj), u3*v3/f, -(f+(1/f)*u3**2), v3],
                   [0, -f*(1/camera_to_obj), v3*(1/camera_to_obj), f+(1/f)*v3**2, -(1/f)*u3*v3, -u3],
                   [-f*(1/camera_to_obj), 0, u4*(1/camera_to_obj), u4*v4/f, -(f+(1/f)*u4**2), v4],
                   [0, -f*(1/camera_to_obj), v4*(1/camera_to_obj), f+(1/f)*v4**2, -(1/f)*u4*v4, -u4]])
    return Ji

def detect_ar_marker_corner_pos_and_depth(marker_size, aruco_dict, parameters, rgb_img, camera_matrix, dist_coeffs):
    """
    ARマーカの4隅の位置とカメラまでの距離を検出する

    Parameters
    ----------
    marker_size : float
        ARマーカの1辺の長さ（メートル）
    aruco_dict : cv2.aruco.Dictionary
        ARマーカの辞書
    parameters : cv2.aruco.DetectorParameters
        ARマーカ検出のパラメータ
    rgb_img : numpy.ndarray
        カメラ画像（RGB）
    camera_matrix : numpy.ndarray
        カメラの内部パラメータ行列
    dist_coeffs : numpy.ndarray
        歪み係数

    Returns
    -------
    corner_pos : numpy.ndarray
        ARマーカの4つの角の位置(x1, y1, x2, y2, x3, y3, x4, y4)[pixel位置]
    depth : float
        カメラからARマーカまでの距離[m]
    """

    # カメラ画像をグレースケールに変換
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)

    # ARマーカの検出
    corners, ids, _ = cv2.aruco.detectMarkers(rgb_img, aruco_dict, parameters=parameters)

    # 検出されたマーカがない場合はNoneを返す
    if ids is None or len(ids) == 0:
        return None, None, None

    # 最初に検出されたマーカの4隅の位置を取得
    marker_corners = corners[0][0]

    # 各隅の位置と対応する深度を取得
    corner_pos = np.zeros(8)
    for i in range(4):
        x, y = int(marker_corners[i][0]), int(marker_corners[i][1])
        # depth = depthImg[y, x]
        corner_pos[i*2] = x
        corner_pos[i*2+1] = y

    # aolvePnPでカメラからARマーカまでの距離を求める
    # 3次元座標
    marker_size = 0.1
    corner_points_3d = np.array([[-marker_size/2, -marker_size/2, 0],
                     [marker_size/2, -marker_size/2, 0],
                     [marker_size/2, marker_size/2, 0],
                     [-marker_size/2, marker_size/2, 0]], dtype=np.float32)
    # 2次元座標
    corner_points_2d = np.array([[corner_pos[0], corner_pos[1]],
                             [corner_pos[2], corner_pos[3]],
                             [corner_pos[4], corner_pos[5]],
                             [corner_pos[6], corner_pos[7]],], dtype=np.float32)

    # solvePnPでカメラからARマーカまでの距離を求める
    _, _, translation_vector = cv2.solvePnP(corner_points_3d, corner_points_2d, camera_matrix, dist_coeffs)

    # カメラからARマーカまでの距離を求める
    depth = translation_vector[2][0]

    return corner_pos, depth



# <a id='toc8_'></a>[カメラのパラメータ設定](#toc0_)

In [7]:
# カメラ設定
fov = 60
image_width = 224
image_height = 224
center_x = image_width / 2
center_y = image_height / 2
aspect = image_width / image_height
near = 0.05
far = 10
projection_matrix = pybullet.computeProjectionMatrixFOV(fov, aspect, near, far)

# 焦点距離を求める
fov_rad = np.deg2rad(fov)
f = image_height / (2 * np.tan(fov_rad / 2))

# カメラの内部パラメータ
camera_matrix = np.array([[f, 0, image_width/2],
                         [0, f, image_height/2],
                         [0, 0, 1]], dtype=np.float32)

# 歪み係数（ここでは、歪みがないと仮定）
dist_coeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)

# <a id='toc9_'></a>[ARマーカに関するパラメータの設定](#toc0_)

In [8]:
marker_size = 0.1 # ARマーカの1辺の長さ（メートル）

# 検出するARマーカの辞書を定義
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters_create()

# <a id='toc10_'></a>[ロボットアームのパラメータなどの設定](#toc0_)

In [9]:
# 各リンクの長さ
ARM_ORIGIN_X_WORLD = 0.0 # ワールド座標系から見たアームの原点のx座標
ARM_ORIGIN_Y_WORLD = 0.0 # ワールド座標系から見たアームの原点のy座標
ARM_ORIGIN_Z_WORLD = 0.8 # ワールド座標系から見たアームの原点のz座標(「simple6d_arm_with_gripper.urdf」の base_link の z方向の長さ)
LINK1_LENGTH = 0.3 # link1の長さ(「simple6d_arm_with_gripper.urdf」の link1 の z方向の長さ)
LINK2_LENGTH = 0.5 # link2の長さ(「simple6d_arm_with_gripper.urdf」の link2 の z方向の長さ)
LINK3_LENGTH = 0.5 # link3の長さ(「simple6d_arm_with_gripper.urdf」の link3 の z方向の長さ)
LINK4_LENGTH = 0.1 # link4の長さ(「simple6d_arm_with_gripper.urdf」の link4 の z方向の長さ)
LINK5_LENGTH = 0.1 # link5の長さ(「simple6d_arm_with_gripper.urdf」の link5 の z方向の長さ)
LINK6_LENGTH = 0.15 # link6の長さ(「simple6d_arm_with_gripper.urdf」の link6 の z方向の長さ)
CAMERA_Z_SIZE = 0.01 # カメラのz方向のサイズ
CAMERA_X_OFFSET = 0.08 # link6座標系の原点からカメラ座標系の原点までのx方向のオフセット

# ここの値を変えると、結果が変わります#############
# 各リンクの初期角度を定義
link1_angle_deg = -90
link2_angle_deg = 55
link3_angle_deg = 35
link4_angle_deg = 0
link5_angle_deg = 0
link6_angle_deg = 0
feature_param = 10

# 画像上の4つの特徴量の目標位置
goal1_u = center_x - 50
goal1_v = center_y - 50
goal2_u = center_x + 50
goal2_v = center_y - 50
goal3_u = center_x + 50
goal3_v = center_y + 50
goal4_u = center_x - 50
goal4_v = center_y + 50
##############################################

# deg -> radへ変換
link1_angle_rad  = np.deg2rad(link1_angle_deg)
link2_angle_rad  = np.deg2rad(link2_angle_deg)
link3_angle_rad  = np.deg2rad(link3_angle_deg)
link4_angle_rad  = np.deg2rad(link4_angle_deg)
link5_angle_rad  = np.deg2rad(link5_angle_deg)
link6_angle_rad  = np.deg2rad(link6_angle_deg)

# 「4つの特徴量の目標位置」をリストにまとめる
goal_feature_points = np.array([goal1_u, goal1_v, goal2_u, goal2_v, goal3_u, goal3_v, goal4_u, goal4_v])

# ロボットの各関節のインデックス
LINK1_JOINT_IDX = 0
LINK2_JOINT_IDX = 1
LINK3_JOINT_IDX = 2
LINK4_JOINT_IDX = 3
LINK5_JOINT_IDX = 4
LINK6_JOINT_IDX = 5
CAMERA_IDX = 6
CAMERA_TARGET_IDX = 7

# ロボットの各関節の角度を初期化
pybullet.resetJointState(arm_id, LINK1_JOINT_IDX, link1_angle_rad )
pybullet.resetJointState(arm_id, LINK2_JOINT_IDX, link2_angle_rad )
pybullet.resetJointState(arm_id, LINK3_JOINT_IDX, link3_angle_rad )
pybullet.resetJointState(arm_id, LINK4_JOINT_IDX, link4_angle_rad )
pybullet.resetJointState(arm_id, LINK5_JOINT_IDX, link5_angle_rad )
pybullet.resetJointState(arm_id, LINK6_JOINT_IDX, link6_angle_rad )

# <a id='toc11_'></a>[ビジュアルサーボの実行](#toc0_)

右側のスライダーの値を変更することで、ARマーカを貼り付けたボックスの位置・姿勢を変更することができます。

なおデフォルトのロボットの姿勢、ボックスの位置の場合「x, roll, yaw」方向の変更は上手く調整しないと適切に動作しないことがあります。  
そのため、最初は「y, z, pitch」方向の変更をお勧めします。

In [10]:
import threading
import time

# グローバル変数（おまけの章でも使用する変数をglobal変数として定義）
current_feature_points = None # 特徴点の現在位置
rgb_img = None # カメラから取得したRGB画像

def VisualServo():
    global current_feature_points
    global rgb_img

    while True:
        # 「ARマーカが張り付けられたボックス」をスライダーに設定された位置、姿勢に設定
        obj_x = pybullet.readUserDebugParameter(0)
        obj_y = pybullet.readUserDebugParameter(1)
        obj_z = pybullet.readUserDebugParameter(2)
        obj_roll = pybullet.readUserDebugParameter(3)
        obj_pitch = pybullet.readUserDebugParameter(4)
        obj_yaw = pybullet.readUserDebugParameter(5)
        pybullet.resetBasePositionAndOrientation(box_bid, [obj_x, obj_y, obj_z], pybullet.getQuaternionFromEuler([obj_roll, obj_pitch, obj_yaw]))

        # 関節角度を取得
        q1 = pybullet.getJointState(arm_id, LINK1_JOINT_IDX)[0]
        q2 = pybullet.getJointState(arm_id, LINK2_JOINT_IDX)[0]
        q3 = pybullet.getJointState(arm_id, LINK3_JOINT_IDX)[0]
        q4 = pybullet.getJointState(arm_id, LINK4_JOINT_IDX)[0]
        q5 = pybullet.getJointState(arm_id, LINK5_JOINT_IDX)[0]
        q6 = pybullet.getJointState(arm_id, LINK6_JOINT_IDX)[0]

        # ロボットアームの手先のカメラの姿勢に合わせて、カメラの上方向のベクトルを設定
        camera_up_vector = np.array([0, -1, 0]) # デフォルトのカメラの上方向のベクトル
        RW1 = Rz(theta=0)@Ry(theta=0)@Rx(theta=0) # ワールド座標系 -> link1座標系
        R12 = Rz(theta=q1) # link1座標系 -> link2座標系
        R23 = Ry(theta=q2) # link2座標系 -> link3座標系
        R34 = Ry(theta=q3) # link3座標系 -> link4座標系
        R45 = Rz(theta=q4) # link4座標系 -> link5座標系
        R56 = Ry(theta=q5) # link5座標系 -> link6座標系
        R6C = Rz(theta=q6-np.pi/2) # link6座標系 -> カメラ座標系（カメラを-90度回転すると、取得される画像が正しい向きになる）
        RWC = RW1@R12@R23@R34@R45@R56@R6C # ワールド座標系 -> カメラ座標系
        rotate_camera_up_vector = RWC@camera_up_vector # カメラ座標系におけるカメラの上方向のベクトル

        # カメラ画像を取得
        camera_link_pose = pybullet.getLinkState(arm_id, CAMERA_IDX)[0] # 手先のカメラリンクの位置
        cameraTargetLinkPose = pybullet.getLinkState(arm_id, CAMERA_TARGET_IDX)[0] # カメラリンクの少しだけ前に設定した仮想的なリンクの位置
        view_matrix = pybullet.computeViewMatrix(cameraEyePosition=[camera_link_pose[0], camera_link_pose[1], camera_link_pose[2]],cameraTargetPosition=[cameraTargetLinkPose[0], cameraTargetLinkPose[1], cameraTargetLinkPose[2]],cameraUpVector=rotate_camera_up_vector)
        _, _, rgb_img, _, _ = pybullet.getCameraImage(
            width=image_width,
            height=image_height,
            viewMatrix=view_matrix,
            projectionMatrix=projection_matrix,
            renderer=pybullet.ER_BULLET_HARDWARE_OPENGL
        )

        # 「画像座標系における4つの特徴量の位置（ピクセル位置）」、「カメラ→オブジェクトの距離[m]」を取得
        current_feature_points, z = detect_ar_marker_corner_pos_and_depth(marker_size, aruco_dict, parameters, rgb_img, camera_matrix, dist_coeffs)

        # 特徴量が検出できなかった場合は、次のループへ
        if current_feature_points is None:
            continue

        # 画像ヤコビ行列
        Ji = make_4features_image_jacobian_matrix(current_feature_points, z, f)
        Ji_inv = np.linalg.pinv(Ji) # 画像ヤコビ行列の逆行列

        # ワールド座標系から見た、各linkの座標系の位置を計算（基礎ヤコビ行列の計算に使用）
        TW1 = Hz(theta=0)@Hy(theta=0)@Hx(theta=0)@Hp(x=ARM_ORIGIN_X_WORLD, y=ARM_ORIGIN_Y_WORLD, z=ARM_ORIGIN_Z_WORLD) # ワールド座標系 -> link1座標系
        T12 = Hz(theta=q1)@Hp(x=0, y=0, z=LINK1_LENGTH) # リンク1座標系 -> リンク2座標系
        T23 = Hy(theta=q2)@Hp(x=0, y=0, z=LINK2_LENGTH) # リンク2座標系 -> リンク3座標系
        T34 = Hy(theta=q3)@Hp(x=0, y=0, z=LINK3_LENGTH) # リンク3座標系 -> リンク4座標系
        T45 = Hz(theta=q4)@Hp(x=0, y=0, z=LINK4_LENGTH) # リンク4座標系 -> リンク5座標系
        T56 = Hy(theta=q5)@Hp(x=0, y=0, z=LINK5_LENGTH) # リンク5座標系 -> リンク6座標系
        T6C = Hz(theta=q6-np.pi/2)@Hp(x=CAMERA_X_OFFSET, y=0.0, z=LINK6_LENGTH-CAMERA_Z_SIZE/2) # link6座標系 -> カメラ座標系
        origin_pos = np.array([0, 0, 0, 1]) # 原点位置
        origin_link1_world = TW1@origin_pos                         # ワールド座標系から見た、リンク1座標系の原点位置
        origin_link2_world = TW1@T12@origin_pos                     # ワールド座標系から見た、リンク2座標系の原点位置
        origin_link3_world = TW1@T12@T23@origin_pos                 # ワールド座標系から見た、リンク3座標系の原点位置
        origin_link4_world = TW1@T12@T23@T34@origin_pos             # ワールド座標系から見た、リンク4座標系の原点位置
        origin_link5_world = TW1@T12@T23@T34@T45@origin_pos         # ワールド座標系から見た、リンク5座標系の原点位置
        origin_link6_world = TW1@T12@T23@T34@T45@T56@origin_pos     # ワールド座標系から見た、リンク6座標系の原点位置
        origin_linkC_world = TW1@T12@T23@T34@T45@T56@T6C@origin_pos # ワールド座標系から見た、カメラ座標系の原点位置

        # ここから、ビジュアルサーボのメイン処理 ##########################################################
        # 「現在の特徴量の位置」→「目標の特徴量の位置」方向のベクトル
        feature_current_to_goal = goal_feature_points - current_feature_points

        # 「現在の特徴量の位置」→「目標の特徴量の位置」方向に向かう微小変化量
        delta_feature_points = feature_param * feature_current_to_goal

        # 「画像ヤコビ行列の逆行列Ji_inv」を用いて、「特徴点の微小変化量delta_feature_points」から、「カメラの微小変化量（=Δx, Δy, Δz, Δroll, Δpitch, Δyaw）camera_delta_p」を求める
        # （カメラをどのように動かせば、特徴点が目標の位置に移動するか）
        delta_p_camera = Ji_inv @ delta_feature_points

        # Rを対角に持った対角行列
        size = 6
        rotate_diagonal_matrix = np.zeros((size, size))
        for i in range(0, size, 3):
            rotate_diagonal_matrix[i:i+3, i:i+3] = RWC

        # 「回転行列rotate_diagonal_matrix」を用いて「カメラ座標系を基準としたカメラの微小変化量delta_p_camera」から「ワールド座標系を基準としたカメラの微小変化量delta_p_world」に変換する
        delta_p_world = rotate_diagonal_matrix @ delta_p_camera

        # 基礎ヤコビ行列
        Jv = make_6d_jacobian_matrix(R=[R12, R23, R34, R45, R56, R6C],
                                     o1=origin_link1_world[0:3], 
                                     o2=origin_link2_world[0:3], 
                                     o3=origin_link3_world[0:3], 
                                     o4=origin_link4_world[0:3], 
                                     o5=origin_link5_world[0:3], 
                                     o6=origin_link6_world[0:3], 
                                     oc=origin_linkC_world[0:3])
        Jv_inv = np.linalg.pinv(Jv) # 基礎ヤコビ行列の逆行列

        # 「（基礎）ヤコビ行列の逆行列Jv_inv」を用いて、「ワールド座標系を基準としたカメラの微小変化量delta_p_world」から、「各関節の角速度delta_q」を求める
        # （ロボットアームの各関節をどのように動かせば、カメラが目標の位置、姿勢に移動するか）
        delta_q = Jv_inv @ delta_p_world

        # 求めた角速度を各関節にセット
        pybullet.setJointMotorControl2(arm_id, LINK1_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[0])
        pybullet.setJointMotorControl2(arm_id, LINK2_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[1])
        pybullet.setJointMotorControl2(arm_id, LINK3_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[2])
        pybullet.setJointMotorControl2(arm_id, LINK4_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[3])
        pybullet.setJointMotorControl2(arm_id, LINK5_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[4])
        pybullet.setJointMotorControl2(arm_id, LINK6_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[5])

        # 1時刻分だけ進める
        pybullet.stepSimulation()
        time.sleep(time_step)

# スレッドを立ち上げる
thread = threading.Thread(target=VisualServo)
thread.start()

# <a id='toc12_'></a>[（おまけ）特徴点位置の描画](#toc0_)

下コードを実行すると、「ゴールの特徴点位置」と「現在の特徴点位置」が描画されます。

具体的に、特徴点がどのように移動しているのかを確認することができます。

In [11]:
import matplotlib.pyplot as plt
%matplotlib qt

plt.ion()  # インタラクティブモードを有効にする
fig, ax = plt.subplots()
goal_plot, = ax.plot([], [], 'ro', label="Goal Points")
current_plot, = ax.plot([], [], 'bo', label="Current Points")
ax.set_xlim(0, image_width)
ax.set_ylim(0, image_height)
ax.invert_yaxis()  # 画像座標系と一致させるためにy軸を反転

plt.legend()
plt.show()

# メインループで、カメラから取得した画像を描画
initDraw = True
while True:
    # rgbImgを描画し、goal_feature_pointsとcurrent_feature_pointsを描画
    try:
        if goal_feature_points is not None and current_feature_points is not None:
            if initDraw:
                img_plot = ax.imshow(rgb_img)
                initDraw = False
            else:
                img_plot.set_data(rgb_img)
            goal_plot.set_xdata([goal1_u, goal2_u, goal3_u, goal4_u])
            goal_plot.set_ydata([goal1_v, goal2_v, goal3_v, goal4_v])
            current_plot.set_xdata([current_feature_points[0], current_feature_points[2], current_feature_points[4], current_feature_points[6]])
            current_plot.set_ydata([current_feature_points[1], current_feature_points[3], current_feature_points[5], current_feature_points[7]])
            plt.pause(0.001)
    except:
        pass